##### Copyright 2022 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Regresión logística para clasificaciones binarias con las API Core

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/core/logistic_regression_core"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/guide/core/logistic_regression_core.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/guide/core/logistic_regression_core.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver el código fuente en GitHub</a> </td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/guide/core/logistic_regression_core.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a></td>
</table>

En esta guía se muestra cómo utilizar las [API de bajo nivel de TensorFlow Core](https://www.tensorflow.org/guide/core) para realizar [clasificación binaria](https://developers.google.com/machine-learning/glossary#binary_classification){:.external} con [regresión logística](https://developers.google.com/machine-learning/crash-course/logistic-regression/){:.external}. Este utiliza el conjunto de datos [Wisconsin Breast Cancer Dataset](https://archive.ics.uci.edu/ml/datasets/breast+cancer+wisconsin+(original)){:.external} para la clasificación de tumores.

[La regresión logística](https://developers.google.com/machine-learning/crash-course/logistic-regression/){:.external} es uno de los algoritmos más populares para la clasificación binaria. Dado un conjunto de ejemplos con características, el objetivo de la regresión logística es obtener valores entre 0 y 1, que pueden interpretarse como las probabilidades de que cada ejemplo pertenezca a una clase concreta. 

## Preparación

Este tutorial utiliza [pandas](https://pandas.pydata.org){:.external} para leer un archivo CSV en un [DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html){:.external}, [seaborn](https://seaborn.pydata.org){:.external} para trazar una relación por pares en un conjunto de datos, [Scikit-learn](https://scikit-learn.org/){:.external} para calcular una matriz de confusión, y [matplotlib](https://matplotlib.org/){:.external} para crear visualizaciones.

In [ ]:
!pip install -q seaborn

In [ ]:
import tensorflow as tf
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
import sklearn.metrics as sk_metrics
import tempfile
import os

# Preset matplotlib figure sizes.
matplotlib.rcParams['figure.figsize'] = [9, 6]

print(tf.__version__)
# To make the results reproducible, set the random seed value.
tf.random.set_seed(22)

## Cargar los datos

A continuación, cargue el conjunto de datos [Wisconsin Breast Cancer Dataset](https://archive.ics.uci.edu/ml/datasets/breast+cancer+wisconsin+(original)){:.external} del repositorio [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/){:.external}. Este conjunto de datos contiene varias características, como el radio, la textura y la concavidad de un tumor.

In [ ]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data'

features = ['radius', 'texture', 'perimeter', 'area', 'smoothness', 'compactness',
            'concavity', 'concave_poinits', 'symmetry', 'fractal_dimension']
column_names = ['id', 'diagnosis']

for attr in ['mean', 'ste', 'largest']:
  for feature in features:
    column_names.append(feature + "_" + attr)

Lea el conjunto de datos en un [DataFrame](){:.externo} de pandas utilizando [`pandas.read_csv`](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html){:.externo}:

In [ ]:
dataset = pd.read_csv(url, names=column_names)

In [ ]:
dataset.info()

Muestra las cinco primeras filas:

In [ ]:
dataset.head()

Divida el conjunto de datos en conjuntos de entrenamiento y prueba utilizando [`pandas.DataFrame.sample`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sample.html){:.external}, [`pandas.DataFrame.drop`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html){:.external} y [`pandas.DataFrame.iloc`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.iloc.html){:.external}. Asegúrese de separar las características de las etiquetas del objetivo. El conjunto de prueba se utiliza para evaluar la generalizabilidad de su modelo a los datos que no se hayan visto.

In [ ]:
train_dataset = dataset.sample(frac=0.75, random_state=1)

In [ ]:
len(train_dataset)

In [ ]:
test_dataset = dataset.drop(train_dataset.index)

In [ ]:
len(test_dataset)

In [ ]:
# The `id` column can be dropped since each row is unique
x_train, y_train = train_dataset.iloc[:, 2:], train_dataset.iloc[:, 1]
x_test, y_test = test_dataset.iloc[:, 2:], test_dataset.iloc[:, 1]

## Tratamiento previo de los datos

Este conjunto de datos contiene el promedio, el error estándar y los valores más grandes para cada una de las 10 medidas del tumor recogidas por el ejemplo. La columna objetivo `"diagnosis"` es una variable categórica con `"M"` lo cual indica un tumor maligno y `"B"` lo cual indica un diagnóstico de tumor benigno. Esta columna debe convertirse a un formato numérico binario para el entrenamiento del modelo.

La función [`pandas.Series.map`](https://pandas.pydata.org/docs/reference/api/pandas.Series.map.html){:.external} es útil para asignar valores binarios a las categorías.

El conjunto de datos también debe convertirse en un tensor con la función `tf.convert_to_tensor` una vez que se haya completado el preprocesamiento.

In [ ]:
y_train, y_test = y_train.map({'B': 0, 'M': 1}), y_test.map({'B': 0, 'M': 1})
x_train, y_train = tf.convert_to_tensor(x_train, dtype=tf.float32), tf.convert_to_tensor(y_train, dtype=tf.float32)
x_test, y_test = tf.convert_to_tensor(x_test, dtype=tf.float32), tf.convert_to_tensor(y_test, dtype=tf.float32)

Utilice [`seaborn.pairplot`](https://seaborn.pydata.org/generated/seaborn.pairplot.html){:.external} para revisar la distribución conjunta de algunos pares de características basadas en el promedio del conjunto de entrenamiento y observe cómo se relacionan con el objetivo:

In [ ]:
sns.pairplot(train_dataset.iloc[:, 1:6], hue = 'diagnosis', diag_kind='kde');

En este diagrama de pares se observa que determinadas características, como el radio, el perímetro y el área, están muy correlacionadas. Esto es de esperarse ya que el radio del tumor está directamente involucrado en el cálculo tanto del perímetro como del área. Además, hay que tener en cuenta que los diagnósticos malignos parecen estar más sesgados a la derecha en muchas de las características.

Asegúrese también de revisar las estadísticas generales. Observe cómo cada característica cubre una gama de valores muy diferente.

In [ ]:
train_dataset.describe().transpose()[:10]

Debido a los rangos inconsistentes, es aconsejable normalizar los datos de manera que cada característica tenga un promedio cero y una varianza igual a uno. Este proceso se denomina [normalización](https://developers.google.com/machine-learning/glossary#normalization){:.external}.

In [ ]:
class Normalize(tf.Module):
  def __init__(self, x):
    # Initialize the mean and standard deviation for normalization
    self.mean = tf.Variable(tf.math.reduce_mean(x, axis=0))
    self.std = tf.Variable(tf.math.reduce_std(x, axis=0))

  def norm(self, x):
    # Normalize the input
    return (x - self.mean)/self.std

  def unnorm(self, x):
    # Unnormalize the input
    return (x * self.std) + self.mean

norm_x = Normalize(x_train)
x_train_norm, x_test_norm = norm_x.norm(x_train), norm_x.norm(x_test)

## Regresión logística

Antes de construir un modelo de regresión logística, es crucial comprender las diferencias entre este método y la regresión lineal tradicional.

### Fundamentos de la regresión logística

La regresión lineal devuelve una combinación lineal de sus entradas; esta salida es ilimitada. La salida de una [regresión logística](https://developers.google.com/machine-learning/glossary#logistic_regression){:.external} está en el rango `(0, 1)`. Para cada ejemplo, representa la probabilidad de que el ejemplo pertenezca a la clase *positiva*.

La regresión logística transforma los resultados continuos de la regresión lineal tradicional, `(-∞, ∞)`, en las probabilidades, `(0, 1)`. Esta transformación también es simétrica, de modo que al invertir el signo de la salida lineal se obtiene una probabilidad inversa a la original.

Sea $Y$ la probabilidad de estar en la clase `1` (el tumor es maligno). El mapeo deseado puede lograrse interpretando el resultado de la regresión lineal como la [relación logarítmica de probabilidades](https://developers.google.com/machine-learning/glossary#log-odds){:.external} de estar en la clase `1` frente a la clase `0`:

$$\ln(\frac{Y}{1-Y}) = wX + b$$

Si se establece $wX + b = z$, esta ecuación puede resolverse para $Y$:

$$Y = \frac{e^{z}}{1 + e^{z}} = \frac{1}{1 + e^{-z}}$$

La expresión $\frac{1}{1 + e^{-z}}$ se conoce como la [función sigmoidal](https://developers.google.com/machine-learning/glossary#sigmoid_function){:.external} $\sigma(z)$. Por lo tanto, la ecuación para la regresión logística se puede escribir como $Y = \sigma(wX + b)$.

El conjunto de datos de este tutorial trata una matriz de características de alta dimensión. Por lo tanto, la ecuación anterior debe reescribirse en forma de matriz vectorial de la siguiente manera:

$${\mathrm{Y}} = \sigma({\mathrm{X}}w + b)$$

donde:

- $\underset{m\times 1}{\mathrm{Y}}$: un vector objetivo
- $\underset{m\times n}{\mathrm{X}}$: una matriz de características
- $\underset{n\times 1}w$: un vector de pesos
- $b$: un sesgo
- $\sigma$: una función sigmoidal aplicada a cada elemento del vector de salida

Empiece visualizando la función sigmoidal, que transforma la salida lineal, `(-∞, ∞)`, para que caiga entre `0` y `1`. La función sigmoidal está disponible en `tf.math.sigmoid`.

In [ ]:
x = tf.linspace(-10, 10, 500)
x = tf.cast(x, tf.float32)
f = lambda x : (1/20)*x + 0.6
plt.plot(x, tf.math.sigmoid(x))
plt.ylim((-0.1,1.1))
plt.title("Sigmoid function");

### La función de pérdida logarítmica

La [función de pérdida logarítmica ](https://developers.google.com/machine-learning/glossary#Log_Loss){:.external}, o pérdida de entropía cruzada binaria, es la función de pérdida ideal para resolver un problema de clasificación binaria con regresión logística. Para cada ejemplo, la pérdida logarítmica cuantifica la similitud entre una probabilidad predicha y el valor verdadero del ejemplo. Esta función se determina mediante la siguiente ecuación:

$$L = -\frac{1}{m}\sum_{i=1}^{m}y_i\cdot\log(\hat{y}_i) + (1- y_i)\cdot\log(1 - \hat{y}_i)$$

donde:

- $\hat{y}$: un vector de probabilidades previstas
- $y$: un vector de objetivos verdaderos

Puede utilizar la función `tf.nn.sigmoid_cross_entropy_with_logits` para calcular la pérdida logarítmica. Esta función aplica automáticamente la activación sigmoidal a la salida de la regresión:

In [ ]:
def log_loss(y_pred, y):
  # Compute the log loss function
  ce = tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=y_pred)
  return tf.reduce_mean(ce)

### La regla de actualización del descenso por gradiente

Las API de TensorFlow Core admiten la diferenciación automática con `tf.GradientTape`. Si tiene curiosidad acerca de las matemáticas detrás de la regresión logística [las actualizaciones del gradiente](https://developers.google.com/machine-learning/glossary#gradient_descent){:.external}, a continuación encontrará una breve explicación:

En la ecuación anterior para la pérdida logarítmica, recuerde que cada $\hat{y}_i$ puede ser reescrito en términos de las entradas como $\sigma({\mathrm{X_i}}w + b)$.

El objetivo es encontrar $w^{em0}$ y $b^{/em0}$ que minimicen la pérdida logarítmica:

$$L = -\frac{1}{m}\sum_{i=1}^{m}y_i\cdot\log(\sigma({\mathrm{X_i}}w + b)) + (1- y_i)\cdot\log(1 - \sigma({\mathrm{X_i}}w + b))$$

Si se toma el gradiente $L$ con respecto a $w$, se obtiene lo siguiente:

$$\frac{\partial L}{\partial w} = \frac{1}{m}(\sigma({\mathrm{X}}w + b) - y)X$$

Al tomar el gradiente $L$ con respecto a $b$, se obtiene lo siguiente:

$$\frac{\partial L}{\partial b} = \frac{1}{m}\sum_{i=1}^{m}\sigma({\mathrm{X_i}}w + b) - y_i$$

Ahora, construya el modelo de regresión logística.

In [ ]:
class LogisticRegression(tf.Module):

  def __init__(self):
    self.built = False
    
  def __call__(self, x, train=True):
    # Initialize the model parameters on the first call
    if not self.built:
      # Randomly generate the weights and the bias term
      rand_w = tf.random.uniform(shape=[x.shape[-1], 1], seed=22)
      rand_b = tf.random.uniform(shape=[], seed=22)
      self.w = tf.Variable(rand_w)
      self.b = tf.Variable(rand_b)
      self.built = True
    # Compute the model output
    z = tf.add(tf.matmul(x, self.w), self.b)
    z = tf.squeeze(z, axis=1)
    if train:
      return z
    return tf.sigmoid(z)

Para validar, asegúrese de que el modelo no entrenado produce valores en el rango de `(0, 1)` para un pequeño subconjunto de datos de entrenamiento.

In [ ]:
log_reg = LogisticRegression()

In [ ]:
y_pred = log_reg(x_train_norm[:5], train=False)
y_pred.numpy()

A continuación, escriba una función de precisión para calcular la proporción de las clasificaciones correctas durante el entrenamiento. Con el fin de recuperar las clasificaciones de las probabilidades predichas, establecer un umbral para que todas las probabilidades superiores al umbral pertenezcan a la clase `1`. Se trata de un hiperparámetro configurable que puede establecerse en `0,5` como valor predeterminado.

In [ ]:
def predict_class(y_pred, thresh=0.5):
  # Return a tensor with  `1` if `y_pred` > `0.5`, and `0` otherwise
  return tf.cast(y_pred > thresh, tf.float32)

def accuracy(y_pred, y):
  # Return the proportion of matches between `y_pred` and `y`
  y_pred = tf.math.sigmoid(y_pred)
  y_pred_class = predict_class(y_pred)
  check_equal = tf.cast(y_pred_class == y,tf.float32)
  acc_val = tf.reduce_mean(check_equal)
  return acc_val

### Entrenar al modelo

El uso de minilotes para el entrenamiento proporciona eficiencia en la memoria y una convergencia más rápida. La API `tf.data.Dataset` tiene funciones útiles para el procesamiento por lotes y la mezcla. La API le permite construir canalizaciones de entrada complejas a partir de piezas sencillas y reutilizables. 

In [ ]:
batch_size = 64
train_dataset = tf.data.Dataset.from_tensor_slices((x_train_norm, y_train))
train_dataset = train_dataset.shuffle(buffer_size=x_train.shape[0]).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test_norm, y_test))
test_dataset = test_dataset.shuffle(buffer_size=x_test.shape[0]).batch(batch_size)

Ahora escriba un bucle de entrenamiento para el modelo de regresión logística. El bucle utiliza la función de pérdida logarítmica y sus gradientes con respecto a la entrada para actualizar repetidamente los parámetros del modelo.

In [ ]:
# Set training parameters
epochs = 200
learning_rate = 0.01
train_losses, test_losses = [], []
train_accs, test_accs = [], []

# Set up the training loop and begin training
for epoch in range(epochs):
  batch_losses_train, batch_accs_train = [], []
  batch_losses_test, batch_accs_test = [], []

  # Iterate over the training data
  for x_batch, y_batch in train_dataset:
    with tf.GradientTape() as tape:
      y_pred_batch = log_reg(x_batch)
      batch_loss = log_loss(y_pred_batch, y_batch)
    batch_acc = accuracy(y_pred_batch, y_batch)
    # Update the parameters with respect to the gradient calculations
    grads = tape.gradient(batch_loss, log_reg.variables)
    for g,v in zip(grads, log_reg.variables):
      v.assign_sub(learning_rate * g)
    # Keep track of batch-level training performance
    batch_losses_train.append(batch_loss)
    batch_accs_train.append(batch_acc)

  # Iterate over the testing data
  for x_batch, y_batch in test_dataset:
    y_pred_batch = log_reg(x_batch)
    batch_loss = log_loss(y_pred_batch, y_batch)
    batch_acc = accuracy(y_pred_batch, y_batch)
    # Keep track of batch-level testing performance
    batch_losses_test.append(batch_loss)
    batch_accs_test.append(batch_acc)

  # Keep track of epoch-level model performance
  train_loss, train_acc = tf.reduce_mean(batch_losses_train), tf.reduce_mean(batch_accs_train)
  test_loss, test_acc = tf.reduce_mean(batch_losses_test), tf.reduce_mean(batch_accs_test)
  train_losses.append(train_loss)
  train_accs.append(train_acc)
  test_losses.append(test_loss)
  test_accs.append(test_acc)
  if epoch % 20 == 0:
    print(f"Epoch: {epoch}, Training log loss: {train_loss:.3f}")

### Evaluación del desempeño

Observe los cambios en la pérdida y precisión de su modelo a lo largo del tiempo. 

In [ ]:
plt.plot(range(epochs), train_losses, label = "Training loss")
plt.plot(range(epochs), test_losses, label = "Testing loss")
plt.xlabel("Epoch")
plt.ylabel("Log loss")
plt.legend()
plt.title("Log loss vs training iterations");

In [ ]:
plt.plot(range(epochs), train_accs, label = "Training accuracy")
plt.plot(range(epochs), test_accs, label = "Testing accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy (%)")
plt.legend()
plt.title("Accuracy vs training iterations");

In [ ]:
print(f"Final training log loss: {train_losses[-1]:.3f}")
print(f"Final testing log Loss: {test_losses[-1]:.3f}")

In [ ]:
print(f"Final training accuracy: {train_accs[-1]:.3f}")
print(f"Final testing accuracy: {test_accs[-1]:.3f}")

El modelo demuestra una alta precisión y una baja pérdida cuando se trata de clasificar tumores en el conjunto de datos de entrenamiento y también generaliza bien a los datos de prueba que no se han visualizado. Para ir un paso más allá, puede explorar las tasas de error que ofrecen más información que la puntuación de precisión general. Las dos tasas de error más populares para los problemas de clasificación binaria son la tasa de falsos positivos (FPR) y la tasa de falsos negativos (FNR).

Para este problema, el FPR es la proporción de predicciones de tumores malignos entre los tumores que son realmente benignos. A la inversa, el FNR es la proporción de predicciones de tumores benignos entre los tumores que son realmente malignos.

Calcule una matriz de confusión utilizando [`sklearn.metrics.confusion_matrix`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html#sklearn.metrics.confusion_matrix){:.external}, la cual evalúa la precisión de la clasificación, y utilice matplotlib para mostrar la matriz:

In [ ]:
def show_confusion_matrix(y, y_classes, typ):
  # Compute the confusion matrix and normalize it
  plt.figure(figsize=(10,10))
  confusion = sk_metrics.confusion_matrix(y.numpy(), y_classes.numpy())
  confusion_normalized = confusion / confusion.sum(axis=1, keepdims=True)
  axis_labels = range(2)
  ax = sns.heatmap(
      confusion_normalized, xticklabels=axis_labels, yticklabels=axis_labels,
      cmap='Blues', annot=True, fmt='.4f', square=True)
  plt.title(f"Confusion matrix: {typ}")
  plt.ylabel("True label")
  plt.xlabel("Predicted label")

y_pred_train, y_pred_test = log_reg(x_train_norm, train=False), log_reg(x_test_norm, train=False)
train_classes, test_classes = predict_class(y_pred_train), predict_class(y_pred_test)

In [ ]:
show_confusion_matrix(y_train, train_classes, 'Training')

In [ ]:
show_confusion_matrix(y_test, test_classes, 'Testing')

Observe las medidas de la tasa de error e interprete su significado en el contexto de este ejemplo. En muchos estudios de pruebas médicas, como la detección del cáncer, tener una tasa de falsos positivos alta para garantizar una tasa de falsos negativos baja es perfectamente aceptable y, de hecho, se fomenta, ya que el riesgo de no diagnosticar un tumor maligno (falso negativo) es mucho peor que clasificar erróneamente un tumor benigno como maligno (falso positivo).

Para controlar el FPR y el FNR, pruebe a cambiar el hiperparámetro umbral antes de clasificar las predicciones de probabilidad. Un umbral más bajo aumenta las posibilidades generales del modelo de realizar una clasificación de tumor maligno. Esto aumenta inevitablemente el número de falsos positivos y el FPR, pero también ayuda a disminuir el número de falsos negativos y el FNR.

## Guardar el modelo

Comience por crear un módulo de exportación que reciba los datos sin procesar y realice las siguientes operaciones:

- Normalización
- Predicción de probabilidades
- Predicción de clases


In [ ]:
class ExportModule(tf.Module):
  def __init__(self, model, norm_x, class_pred):
    # Initialize pre- and post-processing functions
    self.model = model
    self.norm_x = norm_x
    self.class_pred = class_pred

  @tf.function(input_signature=[tf.TensorSpec(shape=[None, None], dtype=tf.float32)])
  def __call__(self, x):
    # Run the `ExportModule` for new data points
    x = self.norm_x.norm(x)
    y = self.model(x, train=False)
    y = self.class_pred(y)
    return y 

In [ ]:
log_reg_export = ExportModule(model=log_reg,
                              norm_x=norm_x,
                              class_pred=predict_class)

Si desea guardar el modelo en su estado actual, puede hacerlo con la función `tf.saved_model.save`. Para cargar un modelo guardado y realizar predicciones, utilice la función `tf.saved_model.load`.

In [ ]:
models = tempfile.mkdtemp()
save_path = os.path.join(models, 'log_reg_export')
tf.saved_model.save(log_reg_export, save_path)

In [ ]:
log_reg_loaded = tf.saved_model.load(save_path)
test_preds = log_reg_loaded(x_test)
test_preds[:10].numpy()

## Conclusión

En este bloc de notas se presentaron algunas técnicas para tratar un problema de regresión logística. Aquí encontrará algunos consejos más que pueden serle útiles:

- Las [API de TensorFlow Core](https://www.tensorflow.org/guide/core) se pueden utilizar para crear flujos de trabajo de aprendizaje automático con altos niveles de configuración.
- El análisis de las tasas de error es una forma excelente de obtener más información sobre el rendimiento de un modelo de clasificación, más allá de su puntuación de precisión global.
- El sobreajuste es otro problema común para los modelos de regresión logística, aunque no fue un problema para este tutorial. Visite el tutorial [Sobreajuste y subajuste](../../tutorials/keras/overfit_and_underfit.ipynb) para obtener más ayuda al respecto.

Para obtener más ejemplos sobre el uso de las API de TensorFlow Core, consulte la [guía](https://www.tensorflow.org/guide/core). Si desea obtener más información sobre la carga y preparación de datos, consulte los tutoriales sobre la [carga de datos de imagen](../../tutorials/load_data/images.ipynb) o la [carga de datos CSV](../../tutorials/load_data/csv.ipynb).